In [52]:
import sys
import time
import re
import argparse
import logging
import json
import datetime as dt

import praw
import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy.sql import Insert
from sqlalchemy_declarative import User, Comment, Post, Subreddit, Base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import exists

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# File Settings
reddit_auth_file = 'reddit_auth.txt'
db_settings_file = 'db_settings.txt'
sub_list_file = 'subs.txt'


def init_sessions():
    """Initializes a PRAW Reddit and a DB connection with their respective credentials
    
    :return: 
    Returns a PRAW Reddit instance and a DB connection session
    """
    # Load Authentication Details
    with open(reddit_auth_file, 'r') as f:
        auth = json.load(f)

    with open(db_settings_file, 'r') as f:
        db_settings = json.load(f)

    usr_agnt = 'python:front_page_praw:v0.2 (by /u/c_nor)'
    reddit = praw.Reddit(client_id=auth['client_id'],
                         client_secret=auth['client_secret'],
                         user_agent=usr_agnt)

    # Create Postgres Connection
    engine_address = 'postgresql://{user}:{password}@{address}/{db_name}'.format(
        user=db_settings['user'],
        password=db_settings['password'],
        address=db_settings['address'],
        db_name=db_settings['db_name']

    )
    cnx = create_engine(engine_address)
    Base.metadata.bind = cnx
    DBSession = sessionmaker(bind=cnx)
    return reddit, DBSession()

In [53]:
reddit, session = init_sessions()

In [ ]:
def store_posts(posts, session, comment_limit=100, more_comments=0, threshold=0, verbose=True):
    """Takes a list of posts, requests the data from the API, and stores it into the database.
    
    :param posts: 
    :param session: 
    :param comment_limit: 
    Maximum number of comments to retrieve from post
    :param more_comments: 
    How many 'More Comments' requests are sent to reddit.
    :param threshold: 
    Threshold number of comments required to request "More Comments".
    :param verbose: 
    Display progress
    :return: 
    """

    # logging
    st_time = time.time()
    posts_completed = 0

    for i, post in enumerate(posts):
        post_id = post.fullname

        # Check to see if post already exists in database
        post_exist = session.query(exists().where(Post.post_id == post_id)).scalar()
        if not post_exist:
            if verbose == True:
                print('Retrieving {0} at {1}. Current step {2} / {3}'.format(post_id,
                                                                             time.strftime('%H:%M'), i + 1, len(posts)))
                print('Elapsed time {}'.format(time.time() - st_time))
                print('Posts completed {}'.format(posts_completed))

            new_post = create_post(post)

            logging.debug('Checking existance of subreddit')
            # If a subreddit is not found in the database, insert it.
            sub_exists = session.query(exists().where(Subreddit.sub_id == new_post.sub_id)).scalar()
            if not sub_exists:
                session.add(create_subreddit(post.subreddit))
                session.commit()

            logging.debug('Checking existance of post author')
            # Insert new user if it's not in Users
            user_exists = session.query(exists().where(User.user_id == new_post.user_id)).scalar()
            if not user_exists:
                session.add(create_user(post))
                session.commit()

            session.add(new_post)
            session.commit()

            # Get list of comments
            logging.debug('Getting comment list')
            post.comments.replace_more(limit=more_comments, threshold=threshold)
            comment_list = post.comments.list()
            comment_total = len(comment_list)
            if comment_total > comment_limit:
                comment_list = comment_list[:comment_limit]
                comment_total = len(comment_list)

            sub_displayname = post.subreddit.display_name

            # User group query
            logger.debug('Checking users')
            post_user_list = set()
            for comment in comment_list:
                try:
                    post_user_list.add(comment.author_fullname)
                except:
                    continue

            logger.debug(post_user_list)
            logger.debug('Querying DB for existing users')

            existing_ids = [x.user_id for x in session.query(User).filter(User.user_id.in_(post_user_list)).all()]
            # Create users that don't exist

            logger.debug('Creating new users')
            users_added = []
            for comment in comment_list:
                try:
                    user_id = comment.author_fullname
                    if (user_id in existing_ids) or (user_id in users_added):
                        pass
                    else:
                        users_added.append(user_id)
                        session.add(create_user(comment))
                except:
                    continue
            session.commit()

            logger.debug('Storing comments')
            for j, comment in enumerate(comment_list):
                logger.debug('Storing comment {0} out of {1}'.format(j, comment_total))
                session.add(create_comment(comment, new_post))
            session.commit()
            posts_completed += 1
        else:
            print('Post {} already exists'.format(post_id))
    print('Subreddit completed in ', str(time.time() - st_time))

In [ ]:
def create_user(e, store_karma = False):
    """
    
    :param usr: 
    :return: 
    """
    if store_karma:
        if type(e) == praw.models.reddit.redditor.Redditor:
            new_user = User(user_id=e.author.fullname)
            new_user.user_name = e.author.name
            new_user.link_karma = e.author.link_karma
            new_user.comment_karma = e.author.comment_karma
    else:
        new_user = User(user_id=e.author_fullname)
        new_user.user_name = e.author.name
        new_user.link_karma = None
        new_user.comment_karma = None
    return new_user


def create_subreddit(sub):
    """
    Constructs an instance of the SQL base class Subreddit to insert into the subreddit table.
    :param sub: 
    :type: Subreddit
    :return: 
    Subreddit object
    """
    new_sub = Subreddit(sub_id=sub.fullname)
    new_sub.sub_name = sub.display_name
    new_sub.created = dt.datetime.fromtimestamp(sub.created_utc)
    new_sub.description = sub.description
    new_sub.sub_count = sub.subscribers
    new_sub.audience = sub.audience_target
    new_sub.url = sub.url
    # print('Create subreddit took {}'.format(time.time() - start_time))

    return new_sub


def create_post(post):
    """
    Constructs an instance of the SQL base class Subreddit to insert into the subreddit table.
    :param subred: 
    :type: Subreddit
    :return: 
    Subreddit object
    """
    new_post = Post(post_id=post.fullname)

    try:
        new_post.user_id = post.author_fullname
    except:
        new_post.user_id = 't2_nan'

    new_post.sub_id = post.subreddit.fullname
    new_post.title = post.title
    new_post.created = dt.datetime.fromtimestamp(post.created_utc)
    new_post.score = post.score
    new_post.target_url = post.url
    new_post.permalink = post.permalink
    new_post.body = post.selftext
    # print('Create post took {}'.format(time.time() - start_time))
    return new_post


def create_comment(comment, post):
    """
    Constructs an instance of the SQL base class Subreddit to insert into the subreddit table.
    :param subred: 
    :type: Subreddit
    :return: 
    Subreddit object
    """
    start_time = time.time()
    new_comment = Comment(comment_id=comment.fullname)
    try:
        new_comment.user_id = comment.author_fullname
    except:
        new_comment.user_id = 't2_nan'

    new_comment.post_id = post.post_id
    new_comment.created = dt.datetime.fromtimestamp(comment.created_utc)
    new_comment.score = comment.score
    new_comment.gilds = comment.gilded
    new_comment.body = comment.body
    new_comment.permalink = comment.permalink
    # print('Create comment took {}'.format(time.time() - start_time))
    return new_comment

In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

In [ ]:
logger.info('Five')

In [ ]:
reddit, db_session = init_sessions()

try:
    with open(sub_list_file, 'r') as f:
        sub_list = map(str.strip, f.readlines())
except OSError:
    sub_list = None

script_st_time = time.time()
print('Script started at ', time.strftime('%H:%M'))

if sub_list is not None:
    for display_name in sub_list:
        praw_subreddit = reddit.subreddit(display_name)
        print('Current subreddit: {}'.format(praw_subreddit.display_name))
        post_list = list(praw_subreddit.hot(limit=10))
        store_posts(post_list, db_session, 20)
        break


In [ ]:
sub_list

In [65]:
test = reddit.subreddit('funny')

In [66]:
post_list = test.hot(limit=10)

In [67]:
post_list = list(post_list)

In [68]:
post_1 = post_list[4]

In [ ]:
post_1.comments.replace_more

In [ ]:
post_1.comments.replace_more(limit=0, threshold=0)

In [ ]:
forest = post_1.comments

In [ ]:
len(forest)

In [ ]:
list_forest = list(forest)

In [ ]:
comment_1 = list_forest[0]

In [ ]:
comment_1.author.fullname

In [ ]:
comment_1.author_fullname

In [ ]:
vars(comment_1)

In [ ]:
comment_3 = list_forest[2]

In [ ]:
comment_2.author.link_karma

In [ ]:
comment_3.author.fullname

In [ ]:
comment_3.author.name

In [ ]:
comment_4 = list_forest[3]

In [ ]:
comment_4.author.fullname

In [ ]:
vars(post_1)

In [ ]:
type(post_1)

In [ ]:
type(post_1) == praw.models.reddit.submission.Submission

In [ ]:
type(post_1.author)

In [6]:
import datetime as dt

In [17]:
import time

In [19]:
time.time()

1540332792.5198777

In [20]:
with open('lo')

SyntaxError: invalid syntax (<ipython-input-20-7587c476eb5d>, line 1)

In [50]:
db_session

In [26]:
reddit, db_session = init_sessions()

In [27]:
import pandas as pd

In [38]:
comments = pd.read_sql('SELECT * FROM comments', db_session.bind)

In [45]:
comments.memory_usage()

Index               80
comment_id    16346248
user_id       16346248
post_id       16346248
created       16346248
score         16346248
gilds         16346248
body          16346248
permalink     16346248
dtype: int64

In [46]:
import argparse

In [47]:
parser = argparse.ArgumentParser(description='Test')

In [48]:
parser.add_argument()

TypeError: _get_positional_kwargs() missing 1 required positional argument: 'dest'

In [49]:
type(session)

NameError: name 'session' is not defined

In [69]:
vars(post_1)

{'_comments_by_id': {},
 '_fetched': False,
 '_flair': None,
 '_info_params': {},
 '_mod': None,
 '_reddit': <praw.reddit.Reddit at 0x2b1b7532b70>,
 'approved_at_utc': None,
 'approved_by': None,
 'archived': False,
 'author': Redditor(name='All_User_Taken'),
 'author_flair_background_color': None,
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_template_id': None,
 'author_flair_text': None,
 'author_flair_text_color': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_2ccdvmj1',
 'author_patreon_flair': False,
 'banned_at_utc': None,
 'banned_by': None,
 'can_gild': False,
 'can_mod_post': False,
 'category': None,
 'clicked': False,
 'comment_limit': 2048,
 'comment_sort': 'best',
 'content_categories': None,
 'contest_mode': False,
 'created': 1540344059.0,
 'created_utc': 1540315259.0,
 'distinguished': None,
 'domain': 'v.redd.it',
 'downs': 0,
 'edited': False,
 'gilded': 0,
 'gildings': {'gid_1': 0, 'gid_2': 0, 'gid_3': 0},
 'hidden': Fals

In [64]:
vars(post_2)

{'_comments_by_id': {},
 '_fetched': False,
 '_flair': None,
 '_info_params': {},
 '_mod': None,
 '_reddit': <praw.reddit.Reddit at 0x2b1b7532b70>,
 'approved_at_utc': None,
 'approved_by': None,
 'archived': False,
 'author': Redditor(name='Geoorgio'),
 'author_flair_background_color': None,
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_template_id': None,
 'author_flair_text': None,
 'author_flair_text_color': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_2gpu1igu',
 'author_patreon_flair': False,
 'banned_at_utc': None,
 'banned_by': None,
 'can_gild': False,
 'can_mod_post': False,
 'category': None,
 'clicked': False,
 'comment_limit': 2048,
 'comment_sort': 'best',
 'content_categories': None,
 'contest_mode': False,
 'created': 1540333808.0,
 'created_utc': 1540305008.0,
 'distinguished': None,
 'domain': 'self.AMA',
 'downs': 0,
 'edited': False,
 'gilded': 0,
 'gildings': {'gid_1': 0, 'gid_2': 0, 'gid_3': 0},
 'hidden': False,
 'hi